In [1]:
import itertools
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance

from NJODE.data_utils import load_dataset
from NJODE.train import train
from models import get_ckpt_model, NJODE

param_dict: {'epochs': [200], 'batch_size': [200], 'save_every': [5], 'learning_rate': [0.001], 'test_size': [0.2], 'seed': [398], 'hidden_size': [10], 'bias': [True], 'dropout_rate': [0.1], 'ode_nn': [((50, 'tanh'), (50, 'tanh'))], 'readout_nn': [((50, 'tanh'), (50, 'tanh'))], 'enc_nn': [((50, 'tanh'), (50, 'tanh'))], 'use_rnn': [False], 'func_appl_X': [[]], 'solver': ['euler'], 'weight': [0.5], 'weight_decay': [1.0], 'dataset': ['BlackScholes', 'Heston', 'OrnsteinUhlenbeck'], 'dataset_id': [None], 'plot': [True], 'evaluate': [True], 'paths_to_plot': [(0, 1, 2, 3, 4)]}
param_dict: {'epochs': [100], 'batch_size': [20], 'save_every': [10], 'learning_rate': [0.001], 'test_size': [0.2], 'training_size': [200, 400, 800, 1600, 3200, 6400, 12800], 'seed': [398], 'hidden_size': [10], 'bias': [True], 'dropout_rate': [0.1], 'ode_nn': [((10, 'tanh'), (10, 'tanh'))], 'readout_nn': [((10, 'tanh'), (10, 'tanh'))], 'enc_nn': [((10, 'tanh'), (10, 'tanh'))], 'use_rnn': [False], 'func_appl_X': [[]], 's

In [2]:
params_dict = {
    'input_size': 9,
    'epochs': 100,
    'hidden_size': 10,
    'output_size': 9,
    'ode_nn': ((50, "tanh"), (50, "tanh")),
    'readout_nn': ((50, "tanh"), (50, "tanh")),
    'enc_nn': ((50, "tanh"), (50, "tanh")),
    'use_rnn': False,
    'options': {'which_loss': 'standard'},
    "input_coords": np.arange(9),
    "output_coords": np.arange(9)
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model = NJODE(**params_dict).to(device)
optimizer = torch.optim.Adam(model.parameters())
get_ckpt_model("../data/saved_models/id-24/best_checkpoint/", model, optimizer, device)
model.eval()

using loss: standard
neuralODE use input scaling with tanh
use residual network: input_size=9, output_size=10
use residual network: input_size=10, output_size=9


NJODE(
  (ode_f): ODEFunc(
    (f): Sequential(
      (0): Linear(in_features=21, out_features=50, bias=True)
      (1): Tanh()
      (2): Dropout(p=0, inplace=False)
      (3): Linear(in_features=50, out_features=50, bias=True)
      (4): Tanh()
      (5): Dropout(p=0, inplace=False)
      (6): Linear(in_features=50, out_features=10, bias=True)
    )
  )
  (encoder_map): FFNN(
    (ffnn): Sequential(
      (0): Linear(in_features=9, out_features=50, bias=True)
      (1): Tanh()
      (2): Dropout(p=0, inplace=False)
      (3): Linear(in_features=50, out_features=50, bias=True)
      (4): Tanh()
      (5): Dropout(p=0, inplace=False)
      (6): Linear(in_features=50, out_features=10, bias=True)
    )
  )
  (readout_map): FFNN(
    (ffnn): Sequential(
      (0): Linear(in_features=10, out_features=50, bias=True)
      (1): Tanh()
      (2): Dropout(p=0, inplace=False)
      (3): Linear(in_features=50, out_features=50, bias=True)
      (4): Tanh()
      (5): Dropout(p=0, inplace=False)
 

In [4]:
x = np.load("../data/training_data/OrnsteinUhlenbeckWithCrossTerms-30/data.npy")
x.shape

(300, 9, 101)

In [4]:
def sqrt_matrix(matrix):
    matrix = matrix.detach().numpy()
    min_eigen = min(np.linalg.eigvals(matrix))
    if min_eigen < 0:
        matrix_new = matrix + np.eye(matrix.shape[0]) * 1e-3 - np.eye(matrix.shape[0]) * min_eigen
        return np.linalg.cholesky(matrix_new)
    else:
        return np.linalg.cholesky(matrix)

def generative_step(X_t, mu_t, sigma_t, delta_t):
    delta_Wt = torch.randn_like(X_t) * delta_t**0.5
    return X_t + mu_t * delta_t + delta_Wt @ sigma_t

def generate_features(X):
    batch_size, num_features = X.shape
    features = [X[:, i] for i in range(num_features)]
    new_features = features + [f ** 2 for f in features]
    new_features += [combo[0] * combo[1] for combo in itertools.combinations(features, 2)]
    return torch.stack(new_features, dim=1)

def get_mu_hat(times, time_ptr, obs_idx, delta_t, T, X, start_X, n_obs_ot, i):
    input_size = start_X.size()[1]
    d = int((-3 + np.sqrt(9 + 8 * input_size)) / 2)
    njode = model.get_pred(times, time_ptr, X, obs_idx, delta_t, T, start_X, n_obs_ot)
    njode_pred = njode["pred"]
    njode_time = njode["pred_t"]
    return (njode_pred[-i, :, :d] - start_X[:, :d]) / (njode_time[-i] - njode_time[0])

def get_sigma_hat(times, time_ptr, obs_idx, delta_t, T, X, start_X, n_obs_ot, i):
    input_size = start_X.size()[1]
    d = int((-3 + np.sqrt(9 + 8 * input_size)) / 2)
    X0 = start_X[:, :d]
    njode = model.get_pred(times, time_ptr, X, obs_idx, delta_t, T, start_X, n_obs_ot)
    njode_pred = njode["pred"]
    njode_time = njode["pred_t"]
    sig1 = (construct_matrix(njode_pred[-i, :], d) - construct_matrix(start_X, d)) / (njode_time[-i] - njode_time[0])
    mu_hat = get_mu_hat(times, time_ptr, obs_idx, delta_t, T, X, start_X, n_obs_ot, i)
    sig2 = X0.T @ mu_hat + mu_hat.T @ X0
    return (sig1 - sig2).detach().numpy()

def construct_matrix(output, d):
    XX = torch.zeros((d, d), dtype=output.dtype)
    for i in range(d):
        XX[i, i] = output[:, d + i]
    index = 2 * d
    for i in range(d):
        for j in range(i + 1, d):
            XX[i, j] = output[:, index]
            XX[j, i] = output[:, index]
            index += 1
    return XX

In [6]:
def generate_synthetic_data(num_iterations, delta_t, initial_X):
    times = []
    time_ptr = [0]
    obs_idx = torch.tensor([], dtype=torch.long)
    T = delta_t
    X = initial_X
    start_X = initial_X
    n_obs_ot = torch.tensor([1])
    current_time = 0.0

    for i in range(num_iterations):
        mu_hat = get_mu_hat(times, time_ptr, obs_idx, delta_t, T, X, start_X, n_obs_ot, 1)
        sigma_hat = get_sigma_hat(times, time_ptr, obs_idx, delta_t, T, X, start_X, n_obs_ot, 1)
        vol_hat = sqrt_matrix(torch.from_numpy(sigma_hat))
        X_t = generative_step(start_X[:, :3], mu_hat, vol_hat, delta_t)
        start_X = generate_features(X_t).float()
        current_time += delta_t
        T += delta_t
        times.append(current_time)
        time_ptr = np.append(time_ptr, len(obs_idx))
        obs_idx = torch.cat((obs_idx, torch.arange(start_X.shape[0], dtype=torch.long)))
        X = torch.cat((X, start_X))
        n_obs_ot = torch.cat((n_obs_ot, n_obs_ot[-1].unsqueeze(0) + 1))

    return np.array(times), time_ptr, obs_idx, X, n_obs_ot


In [8]:
initial_X = torch.tensor([[1.0], [1.5], [2.0], [1.0], [1.5**2], [4.0], [1.5], [2.0], [3.0]]).reshape(1, 9)
num_iterations = 100
delta_t = 0.01

times, time_ptr, obs_idx, X, n_obs_ot = generate_synthetic_data(num_iterations, delta_t, initial_X)

AttributeError: 'float' object has no attribute 'astype'